`homework_adult_income_251204.ipynb`에 작성하세요. # 176

https://www.kaggle.com/datasets/wenruliu/adult-income-dataset

데이터셋 정보: Adult Income Dataset은 1994 Census Income 데이터에 기반하며, 개인의 다양한 특성(나이, 학력, 직업 등)을 바탕으로 50,000달러 이상 소득 여부를 예측하세요.

- CatBoost 모델을 사용해서 범주형(Categorical) 특성에 대한 전처리 과정을 단순화하세요. 
- 최적의 하이퍼파라미터를 찾으세요.
- 적절한 평가지표를 통해 모델의 성능을 출력하세요.


In [1]:
!pip install catboost

   ---------------------------------------- 0.0/102.4 MB ? eta -:--:--
    --------------------------------------- 1.3/102.4 MB 6.7 MB/s eta 0:00:16
   - -------------------------------------- 2.6/102.4 MB 6.6 MB/s eta 0:00:16
   - -------------------------------------- 3.9/102.4 MB 6.7 MB/s eta 0:00:15
   -- ------------------------------------- 5.2/102.4 MB 6.6 MB/s eta 0:00:15
   -- ------------------------------------- 7.1/102.4 MB 6.9 MB/s eta 0:00:14
   --- ------------------------------------ 8.7/102.4 MB 7.1 MB/s eta 0:00:14
   ---- ----------------------------------- 10.5/102.4 MB 7.3 MB/s eta 0:00:13
   ---- ----------------------------------- 11.8/102.4 MB 7.2 MB/s eta 0:00:13
   ----- ---------------------------------- 13.4/102.4 MB 7.3 MB/s eta 0:00:13
   ----- ---------------------------------- 15.2/102.4 MB 7.4 MB/s eta 0:00:12
   ------ --------------------------------- 16.8/102.4 MB 7.4 MB/s eta 0:00:12
   ------- -------------------------------- 18.6/102.4 MB 7.6 MB/s

In [6]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score

# 1. 데이터 불러오기
df = pd.read_csv('data/adult.csv')

# 2. 데이터 전처리
# 2-1. 타겟 변수 변환
df['income'] = df['income'].apply(lambda x: 1 if '>50K' in x else 0)

# [수정된 부분] 2-2. 결측치 처리
# CatBoost의 cat_features에는 'NaN'(float)이 들어가면 안 됩니다.
# 따라서 '?'를 NaN으로 바꾸는 대신, 'Unknown'이라는 문자열로 통일합니다.
df.replace('?', 'Unknown', inplace=True)

# 혹시 모를 원래 데이터의 NaN도 문자열로 변환 (안전장치)
df.fillna('Unknown', inplace=True)

# 2-3. 범주형 변수 식별
cat_features = df.select_dtypes(include=['object']).columns.tolist()
print(f"범주형 변수 목록: {cat_features}")

# 2-4. 데이터 분리
X = df.drop('income', axis=1)
y = df['income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. 모델 정의 (CatBoost Native Grid Search 사용)
model = CatBoostClassifier(
    loss_function='Logloss',
    eval_metric='AUC',
    task_type='CPU',
    verbose=0,
    early_stopping_rounds=50,
    cat_features=cat_features # 수정된 범주형 변수 사용
)

# 튜닝할 파라미터 그리드
param_grid = {
    'learning_rate': [0.03, 0.1],
    'depth': [4, 6, 8],
    'l2_leaf_reg': [1, 3, 5]
}

print("최적의 하이퍼파라미터 탐색 중 (CatBoost Built-in Grid Search)...")

# CatBoost 내장 grid_search 사용 (Scikit-learn 호환성 문제 해결)
grid_search_result = model.grid_search(
    param_grid, 
    X=X_train, 
    y=y_train, 
    cv=3, 
    partition_random_seed=42,
    calc_cv_statistics=True, 
    search_by_train_test_split=False,
    refit=True, 
    verbose=False,
    plot=False
)

# 4. 결과 확인
print(f"\n최적 파라미터: {grid_search_result['params']}")

# 5. 성능 평가
# refit=True 덕분에 model은 이미 최적 파라미터로 학습되어 있습니다.
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print("\n========== 모델 성능 평가 ==========")
print(f"Accuracy (정확도): {accuracy_score(y_test, y_pred):.4f}")
print(f"ROC-AUC 점수: {roc_auc_score(y_test, y_proba):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

범주형 변수 목록: ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country']
최적의 하이퍼파라미터 탐색 중 (CatBoost Built-in Grid Search)...
Training on fold [0/3]

bestTest = 0.9232107011
bestIteration = 994

Training on fold [1/3]

bestTest = 0.9284629684
bestIteration = 787

Training on fold [2/3]

bestTest = 0.9286169623
bestIteration = 984

Metric AUC is not calculated on train by default. To calculate this metric on train, add hints=skip_train~false to metric parameters.
Training on fold [0/3]

bestTest = 0.9246350047
bestIteration = 513

Training on fold [1/3]

bestTest = 0.9283948638
bestIteration = 277

Training on fold [2/3]

bestTest = 0.9298165236
bestIteration = 416

Metric AUC is not calculated on train by default. To calculate this metric on train, add hints=skip_train~false to metric parameters.
Training on fold [0/3]

bestTest = 0.9227778746
bestIteration = 999

Training on fold [1/3]

bestTest = 0.9279282825
bestIteration = 998

Traini

KeyboardInterrupt: 